In [2]:
import numpy as np
import pandas as pd

In [3]:
puntos  = pd.read_csv('datos/puntos.csv')

In [4]:
dataset = pd.read_csv('datos/dataset.csv')

In [5]:
merged = pd.merge(dataset, puntos, on='imagen', how='inner')
merged


,imagen,categoria,distancia (mm),punto_referencia_x,punto_referencia_y,punto 1,punto 2,punto 3,punto 4,punto 5,...,punto 13,punto 14,punto 15,distancia,vel alambre,flujo gas,peri voltaje,voltaje,corriente,ubicacion
0,Cordon10_Seccion 1 horizontal C10.jpg,inicio,7.37,435,308,"230,310","226,257","223,193","240,141","266,101",...,"657,225","653,283","659,302",198.002525,5.0,5.0,5.0,5.0,5.0,1
1,Cordon11_Seccion 1 horizontal C11.jpg,inicio,8.17,444,296,"213,297","200,266","199,198","220,161","245,129",...,"645,197","654,254","710,296",189.002645,5.0,5.0,5.0,5.0,5.0,1


In [6]:
coordenadas = merged.drop([
    'imagen','categoria','ubicacion','distancia (mm)',
    'punto_referencia_x', 'punto_referencia_y','distancia',
    'vel alambre', 'flujo gas','peri voltaje','corriente','voltaje',
], axis=1)

In [7]:
coordenadas 

,punto 1,punto 2,punto 3,punto 4,punto 5,punto 6,punto 7,punto 8,punto 9,punto 10,punto 11,punto 12,punto 13,punto 14,punto 15
0,"230,310","226,257","223,193","240,141","266,101","331,57","407,18","494,30","556,47","601,79","647,138","659,172","657,225","653,283","659,302"
1,"213,297","200,266","199,198","220,161","245,129","283,96","334,77","399,60","454,58","505,66","569,100","615,144","645,197","654,254","710,296"


In [8]:
def parse_point(point_str):
    # Separa la cadena en dos valores numéricos
    x, y = map(int, point_str.split(','))
    return x, y

def create_output_grid(puntos_df, grid_rows=8, grid_cols=6):
    grid = np.zeros((grid_rows, grid_cols), dtype=int)

    for _, point_str in puntos_df.items():
        x, y = parse_point(point_str)
        
        col = min(int(x / 1000 * grid_cols), grid_cols - 1) 
        row = min(int(y / 1000 * grid_rows), grid_rows - 1)  
        grid[row, col] = 1

    return grid.flatten()

def create_dataset(df):
    dataset = []
    for _, row in df.iterrows():
        output = create_output_grid(row)
        dataset.append(output)
    return np.array(dataset)


In [9]:
output_dataset = create_dataset(coordenadas)

output_dataset[0]

array([0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [21]:
import cv2

def draw_grid_on_image(image, grid, reference_point, scale_mm_per_pixel, grid_size_mm=(600, 800)):
    ref_x, ref_y = reference_point
    grid_rows, grid_cols = 6, 8  # Cambiado a 6 filas y 8 columnas
    grid = grid.reshape(grid_rows, grid_cols)

    cell_width_px = int(grid_size_mm[0] / grid_cols / scale_mm_per_pixel)
    cell_height_px = int(grid_size_mm[1] / grid_rows / scale_mm_per_pixel)

    # Ajustar el punto de inicio para que el punto de referencia esté en el centro horizontal
    start_x = int(ref_x - (grid_cols * cell_width_px) / 2)
    start_y = ref_y  # El punto de referencia es el 0 en Y

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_color = (255, 255, 255)  # Blanco
    thickness = 1

    for i in range(grid_rows):
        for j in range(grid_cols):
            cell_value = grid[i, j]
            cell_x = start_x + j * cell_width_px
            cell_y = start_y + i * cell_height_px

            # Dibujar el contorno de cada celda
            cv2.rectangle(image, (cell_x, cell_y),
                          (cell_x + cell_width_px, cell_y + cell_height_px),
                          (255, 0, 0), 1)  # Azul, grosor 1

            # Escribir el valor (0 o 1) en el centro de la celda
            text = str(cell_value)
            text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
            text_x = cell_x + (cell_width_px - text_size[0]) // 2
            text_y = cell_y + (cell_height_px + text_size[1]) // 2
            cv2.putText(image, text, (text_x, text_y), font, font_scale, font_color, thickness)

    return image

# Ejemplo de uso:
image = cv2.imread('imagenes/seleccionadas/inicio/Cordon10_Seccion 1 horizontal C10.jpg')
grid = output_dataset[0]
reference_point = (445, 307)  # Punto de referencia (x, y) en la imagen
scale_mm_per_pixel = 3 / 0.72  # Escala: 0.737 mm por pixel

result_image = draw_grid_on_image(image, grid, reference_point, scale_mm_per_pixel)
cv2.imshow('Imagen con grilla', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread (0x2654f80).
Cannot move to target thread (0x1c54ae0)

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread (0x2654f80).
Cannot move to target thread (0x1c54ae0)

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread (0x2654f80).
Cannot move to target thread (0x1c54ae0)

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread (0x2654f80).
Cannot move to target thread (0x1c54ae0)

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread (0x2654f80).
Cannot move to target thread (0x1c54ae0)

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread (0x2654f80).
Cannot move to target thread (0x1c54ae0)

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread (0x2654f80).
Cannot move to target thread (0x1c54ae0)

QObject::moveToThread: Current thread (0x1c54ae0) is not the object's thread